# import 

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn import metrics
import scipy
from scipy import stats
import statsmodels as sm
from statsmodels.distributions.empirical_distribution import ECDF


C:\Users\w10\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# load phat data

In [2]:
phat_path = r'C:\Users\w10\Desktop\웰컴\AS\8주차(데이터추가추출부터)\phat\seg1\ensemble'

In [3]:
os.listdir(phat_path)

['model_valid_phat_df.csv',
 'test_phat_df.csv',
 'train_phat_df.csv',
 'valid_phat_df.csv']

In [4]:
prob_train_df = pd.read_csv(os.path.join(phat_path, 'train_phat_df.csv'))
prob_valid_df = pd.read_csv(os.path.join(phat_path, 'valid_phat_df.csv'))
prob_test_df = pd.read_csv(os.path.join(phat_path, 'test_phat_df.csv'))
prob_model_valid_df = pd.read_csv(os.path.join(phat_path, 'model_valid_phat_df.csv'))

In [5]:
prob_total_df = pd.concat([prob_train_df, prob_valid_df, prob_test_df], axis = 0)

In [6]:
prob_train_df.head(2)

,no,BAD,ensemble_phat
0,1,1,0.467364
1,3,1,0.457814


In [7]:
prob_model_valid_df.head(2)

,no,BAD,ensemble_phat
0,363763,0,0.340212
1,363765,0,0.264771


# load 10grade result

In [8]:
grade_path = r'C:\Users\w10\Desktop\웰컴\AS\8주차(데이터추가추출부터)\모형등급화\seg1'

In [9]:
result_grade = pd.read_excel(os.path.join(grade_path, 'prob_10grade.xlsx'))

In [10]:
result_grade = result_grade.fillna(-np.inf)

In [11]:
result_grade

,grade,p_left,p_right
0,1,-inf,0.156284
1,2,0.156284,0.172464
2,2,0.172464,0.183458
3,2,0.183458,0.192265
4,2,0.192265,0.200017
...,...,...,...
95,9,0.642520,0.654131
96,9,0.654131,0.668138
97,9,0.668138,0.684853
98,10,0.684853,0.707739


In [23]:
result_grade['p_interval'] = result_grade[['p_left','p_right']].apply(lambda x: pd.Interval(left = x['p_left'], right = x['p_right']), axis = 1)

In [24]:
p_interval = pd.Categorical(result_grade['p_interval']).categories

In [25]:
prob_train_df['p_interval'] = pd.cut(prob_train_df['ensemble_phat'], bins = p_interval, duplicates = 'drop')
prob_valid_df['p_interval'] = pd.cut(prob_valid_df['ensemble_phat'], bins = p_interval, duplicates = 'drop')
prob_test_df['p_interval'] = pd.cut(prob_test_df['ensemble_phat'], bins = p_interval, duplicates = 'drop')
prob_model_valid_df['p_interval'] = pd.cut(prob_model_valid_df['ensemble_phat'], bins = p_interval, duplicates = 'drop')
prob_total_df['p_interval'] = pd.cut(prob_total_df['ensemble_phat'], bins = p_interval, duplicates = 'drop')

In [26]:
prob_train_df = prob_train_df.merge(result_grade[['grade','p_interval']], on = ['p_interval'], how = 'inner')
prob_valid_df = prob_valid_df.merge(result_grade[['grade','p_interval']], on = ['p_interval'], how = 'inner')
prob_test_df = prob_test_df.merge(result_grade[['grade','p_interval']], on = ['p_interval'], how = 'inner')
prob_model_valid_df = prob_model_valid_df.merge(result_grade[['grade','p_interval']], on = ['p_interval'], how = 'inner')
prob_total_df = prob_total_df.merge(result_grade[['grade','p_interval']], on = ['p_interval'], how = 'inner')

In [27]:
prob_train_df

,no,BAD,ensemble_phat,p_interval,grade
0,1,1,0.467364,"(0.4653493141602456, 0.4705028915983511]",7
1,227,0,0.470062,"(0.4653493141602456, 0.4705028915983511]",7
2,496,1,0.470126,"(0.4653493141602456, 0.4705028915983511]",7
3,736,0,0.467889,"(0.4653493141602456, 0.4705028915983511]",7
4,942,1,0.468934,"(0.4653493141602456, 0.4705028915983511]",7
...,...,...,...,...,...
73528,416989,0,0.414750,"(0.4128033748683516, 0.418025516917058]",6
73529,417883,0,0.413472,"(0.4128033748683516, 0.418025516917058]",6
73530,417957,0,0.416190,"(0.4128033748683516, 0.418025516917058]",6
73531,418210,0,0.412919,"(0.4128033748683516, 0.418025516917058]",6


In [28]:
def group_grade(data):
    grp_grade = data.groupby(by = ['grade'], as_index = False).agg(
    {
        'BAD' : ('count', 'sum')
    })
    grp_grade.columns = ['grade','Total','Bad', ]
    
    grp_grade['Good'] = grp_grade['Total'] - grp_grade['Bad']
    grp_grade['Bad_rate'] = grp_grade['Bad'] / grp_grade['Total']
    
    
    grp_grade['Distribution Good'] = grp_grade['Good'] / grp_grade['Good'].sum()
    grp_grade['Distribution Bad'] = grp_grade['Bad'] / grp_grade['Bad'].sum()
    grp_grade['Distribution Total'] = grp_grade['Total'] / grp_grade['Total'].sum()
    
    grp_grade['Cum Good'] = np.cumsum(grp_grade['Good'])
    grp_grade['Cum Bad'] = np.cumsum(grp_grade['Bad'])
    grp_grade['Cum Total'] = np.cumsum(grp_grade['Total'])
    
    grp_grade['Cum Dist Good'] = np.cumsum(grp_grade['Good']) / grp_grade['Good'].sum()
    grp_grade['Cum Dist Bad'] = np.cumsum(grp_grade['Bad']) / grp_grade['Bad'].sum()
    grp_grade['Cum Dist Total'] = np.cumsum(grp_grade['Total']) / grp_grade['Total'].sum()
    grp_grade['Cum Bad rate'] = grp_grade['Cum Bad'] / grp_grade['Cum Total']
    
    
    grp_grade['WoE'] = np.log(grp_grade['Distribution Good'] / grp_grade['Distribution Bad'])
    grp_grade = grp_grade.replace({'WoE' : {np.inf : 0, -np.inf : 0}})
    grp_grade['IV'] = (grp_grade['Distribution Good'] - grp_grade['Distribution Bad']) * grp_grade['WoE']
    grp_grade['KS'] = np.abs(grp_grade['Cum Dist Good'] - grp_grade['Cum Dist Bad'])*100

#     grp_grade.reset_index( inplace = True)

#     grp_grade.rename(columns = {'index' : 'grade'}, inplace = True)
#     grp_grade['grade'] = grp_grade['grade'] + 1
    
    return grp_grade

In [29]:
df_list = [prob_train_df, prob_valid_df, prob_test_df, prob_model_valid_df, prob_total_df]
key_list = ['train','valid','test','model_valid','total']

In [30]:
result_dict = dict()
for key, df in zip(key_list, df_list):
    print(key)
    grp = group_grade(df)
    result_dict[key] = grp

train
valid
test
model_valid
total


In [31]:
mv_distribution = result_dict['model_valid']['Distribution Total']

for key, df in result_dict.items():
    if key == 'model_valid':
        pass
    else:
        df['psi'] = (df['Distribution Total'] - mv_distribution) * np.log(df['Distribution Total'] / mv_distribution)

In [32]:
mv_distribution

0    0.018058
1    0.099518
2    0.149278
3    0.159761
4    0.178170
5    0.183788
6    0.119232
7    0.062350
8    0.024428
9    0.005417
Name: Distribution Total, dtype: float64

In [33]:
result_dict['train']

,grade,Total,Bad,Good,Bad_rate,Distribution Good,Distribution Bad,Distribution Total,Cum Good,Cum Bad,Cum Total,Cum Dist Good,Cum Dist Bad,Cum Dist Total,Cum Bad rate,WoE,IV,KS,psi
0,1,736,53,683,0.072011,0.015568,0.001787,0.010009,683,53,736,0.015568,0.001787,0.010009,0.072011,2.164816,0.029835,1.378157,0.004749
1,2,5147,730,4417,0.141830,0.100682,0.024611,0.069996,5100,783,5883,0.116250,0.026397,0.080005,0.133095,1.408785,0.107168,8.985250,0.010389
2,3,8824,1740,7084,0.197189,0.161473,0.058661,0.120001,12184,2523,14707,0.277723,0.085058,0.200005,0.171551,1.012567,0.104105,19.266500,0.006392
3,4,11030,2781,8249,0.252131,0.188029,0.093756,0.150001,20433,5304,25737,0.465752,0.178815,0.350006,0.206085,0.695895,0.065604,28.693722,0.000615
4,5,11765,3706,8059,0.315002,0.183698,0.124941,0.159996,28492,9010,37502,0.649450,0.303756,0.510002,0.240254,0.385450,0.022648,34.569388,0.001955
5,6,12500,5119,7381,0.409520,0.168243,0.172578,0.169992,35873,14129,50002,0.817693,0.476333,0.679994,0.282569,-0.025437,0.000110,34.135943,0.001077
6,7,10295,5541,4754,0.538222,0.108363,0.186805,0.140005,40627,19670,60297,0.926056,0.663138,0.819999,0.326219,-0.544575,0.042717,26.291792,0.003336
7,8,7353,5052,2301,0.687067,0.052449,0.170319,0.099996,42928,24722,67650,0.978505,0.833457,0.919995,0.365440,-1.177827,0.138830,14.504821,0.017783
8,9,4412,3595,817,0.814823,0.018623,0.121199,0.060000,43745,28317,72062,0.997128,0.954656,0.979995,0.392953,-1.873047,0.192130,4.247215,0.031966
9,10,1471,1345,126,0.914344,0.002872,0.045344,0.020005,43871,29662,73533,1.000000,1.000000,1.000000,0.403384,-2.759254,0.117191,0.000000,0.019056


In [34]:
save_path = os.path.join(r'C:\Users\w10\Desktop\웰컴\AS\8주차(데이터추가추출부터)\모형등급화\seg1', 'prob_10grade_result.xlsx')
    
    
with pd.ExcelWriter(save_path,) as writer:
    for key, df in result_dict.items():
        df.to_excel(writer, sheet_name = key, index = False)

In [35]:
prob_train_df = prob_train_df.drop(['p_interval'], axis = 1)
prob_valid_df = prob_valid_df.drop(['p_interval'], axis = 1)
prob_test_df = prob_test_df.drop(['p_interval'], axis = 1)
prob_model_valid_df = prob_model_valid_df.drop(['p_interval'], axis = 1)
prob_total_df = prob_total_df.drop(['p_interval'], axis = 1)

In [36]:
save_path = r'C:\Users\w10\Desktop\웰컴\AS\8주차(데이터추가추출부터)\모형등급화\seg1\data'

prob_train_df.to_csv(os.path.join(save_path, 'train_grade_df.csv'), index = False)
prob_valid_df.to_csv(os.path.join(save_path, 'valid_grade_df.csv'), index = False)
prob_test_df.to_csv(os.path.join(save_path, 'test_grade_df.csv'), index = False)
prob_model_valid_df.to_csv(os.path.join(save_path, 'model_valid_grade_df.csv'), index = False)
prob_total_df.to_csv(os.path.join(save_path, 'total_grade_df.csv'), index = False)

In [37]:
grade_info_df = result_grade.groupby(by = ['grade'], as_index = False).agg(
                {
                    'p_left' : ('min'),
                    'p_right': ('max')
                })

In [38]:
grade_info_df

,grade,p_left,p_right
0,1,-inf,0.156284
1,2,0.156284,0.216992
2,3,0.216992,0.263824
3,4,0.263824,0.313587
4,5,0.313587,0.375890
5,6,0.375890,0.460355
6,7,0.460355,0.538683
7,8,0.538683,0.612551
8,9,0.612551,0.684853
9,10,0.684853,1.000000


In [39]:
save_path = r'C:\Users\w10\Desktop\웰컴\AS\8주차(데이터추가추출부터)\모형등급화\seg1\data'
grade_info_df.to_excel(os.path.join(save_path, 'grade_phat_df.xlsx'), index = False)